In [1]:
(ns day5)
*ns*

#namespace[day5]

In [2]:
(def mem
    (mapv read-string (-> (slurp "data/day05.txt")
(clojure.string/split #","))))
;;mem


#'day5/mem

In [3]:
(defn pow [x n]
    (apply * (repeat n x)))

 (pow 10 2)

100

In [4]:
(defn opcode [{:keys [mem ip]}]
    (mem ip))

(defn param [state n]
    (let [{:keys [mem ip]} state]
    (mem (+ ip n))))

(defn operation [opcode]
           (mod opcode 100))

(defn mode [opcode n] 
    (if ( = 0 (mod 
     (quot opcode 
           (pow 10 (inc n)))
     10)) :pos :imm))
    

(defn paramval [state n]
    (if (= :imm (mode (opcode state) n))
        (param state n)
        ((state :mem) (param state n))
        )
    )


(mode 1200 3)
;; (run mem)


:pos

In [23]:
(defn doinput [state]
    (let [
          {:keys [mem input]} state
          param1 (param state 1)
          mem (assoc mem param1 input)
          ]
        (update (assoc state :mem mem) :ip +  2))
        )
    


#'day5/doinput

In [6]:
(defn dooutput [state]
    (let [
          param1 (paramval state 1)
          ]
        ;;(println newoutput)
        (update (update state :ip + 2)
               :output conj param1 ) 
         
         )
        
        )
    

#'day5/dooutput

In [32]:
(defn doplus [state]
    (let [
        param1 (paramval state 1)
        param2 (paramval state 2)
        target (param state 3)
    
          mem (assoc (state :mem) target (+ param1 param2))
          
          ]
        ;;(println (str target "," param1 "," param2))
        (update (assoc state :mem mem) :ip + 4)
        )
    )

#'day5/doplus

In [25]:
(defn domult [state]
    (let [
        param1 (paramval state 1)
        param2 (paramval state 2)
        target (param state 3)
    
          mem (assoc (state :mem) target (* param1 param2))
          
          ]
        ;;(println (str target "," param1 "," param2))
        (update (assoc state :mem mem) :ip + 4)
        )
    )

#'day5/domult

In [20]:
(defn doequals [state]
    (let [
        param1 (paramval state 1)
        param2 (paramval state 2)
        target (param state 3)
        targetval (if (= param1 param2) 1 0)   
          mem (assoc (state :mem) target targetval)         
          ]
        ;;(println (str target "," param1 "," param2))
        (update (assoc state :mem mem) :ip + 4)
        )
    )

#'day5/doequals

In [26]:
(defn doless [state]
    (let [
        param1 (paramval state 1)
        param2 (paramval state 2)
        target (param state 3)
        targetval (if (< param1 param2) 1 0)  
        mem (assoc (state :mem) target targetval)
          ]
        ;;(println (str target "," param1 "," param2))
        (update (assoc state :mem mem) :ip + 4)
        )
    )

#'day5/doless

In [18]:
(defn dojumptrue [state]
    (let [
        param1 (paramval state 1)
        param2 (paramval state 2)         
          ]
    (if (not (zero? param1))
        (assoc state :ip param2)     
        (update state :ip + 3)
        )
    ))

#'day5/dojumptrue

In [12]:
(defn dojumpfalse [state]
    (let [
    
        param1 (paramval state 1)
        param2 (paramval state 2)
    

          
          ]
    (if  (zero? param1)
        (assoc state :ip param2)     
        (update state :ip + 3)
        )
    ))

#'day5/dojumpfalse

In [13]:


(defn step [state] 
    ;;(println state)
    ;;(println pos)
        (let [
          operation (operation (opcode state))
          ]
              
    (case operation
        1 (doplus state)
        2 (domult state)
        3 (doinput state)
        4 (dooutput state)
        5 (dojumptrue state)
        6 (dojumpfalse state)
        7 (doless state)
        8 (doequals state)
        99 (assoc state :halted true)
        (throw 
         (Exception. 
          (str "unknown op " operation))))))

(defn run [mem input]
    "Run the progam in mem with input and return a sequence of all outputs"
   (:output (last (take-while #(not (:halted %))
     (iterate step {:mem mem :output [] :input input :ip 0 :halted false})))))
     


#'day5/run

In [30]:
(run mem 1)

[0 0 0 0 0 0 0 0 0 13978427]

In [28]:
(last (run mem 1))

13978427

In [34]:
(last (run mem 5))

11189491